In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from math import ceil
import os

In [2]:
nTurn = 1024
nx = 100
pipe_radius = 0.02*2.0**0.5

In [3]:
beam,lattice = impact.readInputFile('test.SF.in')
ke   = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

lattice=[lattice[0],lattice[2]]

betx = lattice[1].betx
NL_t = lattice[1].strength_t
NL_c = lattice[1].transverse_scale_c
lattice[1].length = -lattice[1].length
lattice[1].n_sckick = 50
lattice[1].n_map = 5

reading ImpactZ input file (test.SF.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [4]:
testP = 0.0
sig = 0.01
k=0
q_m = beam.multi_charge.q_m[0]
for x in np.linspace(-0.01,0.01,nx):
    for y in np.linspace(-0.01,0.01,nx):
        if x**2+y**2<=sig**2 :
            k+=1         
            
npt = k

In [5]:
TBT = impact.getElem('TBT')
TBT.file_id = nTurn+100000
lattice.append(TBT)

TBT_integral = impact.getElem('TBT_integral')
TBT_integral.file_id = nTurn+100000
TBT_integral.betx = betx
TBT_integral.alfx = 0.0
TBT_integral.strength_t = NL_t
TBT_integral.transverse_scale_c = NL_c
lattice.append(TBT_integral)

lattice[0].turns = nTurn

for i in range(len(lattice)):
    if 'pipe_radius' in lattice[i].keys() and lattice[i].type!='dipole':
        lattice[i].pipe_radius = pipe_radius
        
lattice.append(impact.getElem('write_raw_ptcl'))
lattice[-1].turn = nTurn
lattice[-1].file_id = -nTurn - 100000

In [6]:
beam.distribution.distribution_type = 'ReadFile_binary'
beam.distribution.file_id = nTurn
beam.n_particles = npt
beam.nCore_y = 6
beam.nCore_z = 4
impact.writeInputFile(beam,lattice,'test.in.'+str(nTurn)+'.backward') 
#impact.run(beam)

In [7]:
with open('test.in.'+str(nTurn)+'.backward') as f:
    testIn=f.read()
print(testIn)

!================= Beam & Control Parameters ================= 
6 4 
6 7668 1 0 1 
64 64 1025 9 0.0338514 0.0338514 0.1 
-23 0 0 1 
7668 
0.0 
1.0657889726792521e-09 
1024 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 
2.2092772151898723 1.065789e-10 0.0 1.0 1.0 0.0 0.0 
0.0 2500000.0 938272045.0 1.0 30000000.0 0.0 
!========================== Lattice ========================== 
0.0 0 0 -16 0.0 1024 / 
-1.8 50 5 9 0.4 0.01 1.2732395447351628 0.028284271247461905 / 
0.0 0 101024 -89 / 
0.0 0 101024 -88 1.2732395447351628 0.0 0.4 0.01 / 
0.0 0 -101024 -2 1024 1 / 

